In [1]:
import os
import scipy
import utils

import numpy as np
import pandas as pd

In [4]:
datadir = '../datasets/NNN/'
fnames = utils.fnames(datadir)
# utils.compute_noise_ceiling(data) # data should be (units, conditions, reps)
dat = pd.read_pickle(os.path.join(datadir, 'fr_data.pkl'))

In [17]:
dat['session'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59])

In [22]:
trials = gus_data['meta_data']['trial_valid_idx'].squeeze()
localizer_imgs = list(trials[trials>1000])

counts = [localizer_imgs.count(i) for i in localizer_imgs]
np.max(counts)

np.int64(7)